In [ ]:
import pandas as pd
import datetime
import os.path
HOUR_PER_DAY = 24

In [ ]:
def _hourly_pivot(df, hour):
    strColumn = "{:02d}".format(hour)
    df_result = df.pivot(index="日期", columns="測項", values=strColumn)
    df_result.index = df_result.index + datetime.timedelta(hours=hour)
    return df_result

In [ ]:
for i in range(11):
    frames = list()
    for j in range(96, 106):
        if os.path.isfile("../data/central_reg/" + str(i+1) + "/" + str(j) + ".csv"):
            df = pd.read_csv("../data/central_reg/" + str(i+1) + "/" + str(j) + ".csv", parse_dates=[0],
                             date_parser = pd.core.tools.datetimes.to_datetime, encoding="big5")
        elif os.path.isfile("../data/central_reg/" + str(i+1) + "/" + str(j) + ".xls"):
            df = pd.read_excel("../data/central_reg/" + str(i+1) + "/" + str(j) + ".xls", parse_dates=[u"日期"])
        else:
            pass
            
        df.columns = ['日期','測站','測項', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09',
                      '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
        
        df = df.replace("NR", 0.0)
        df = pd.concat([_hourly_pivot(df, hour) for hour in range(HOUR_PER_DAY)])
        df.index.name = "datetime"
        df = df.sort_index()
        frames.append(df)
        
    my_output = "../data/central_reg/" + str(i+1) + ".csv"
    dataframe = pd.concat(frames)
    columns = dataframe.columns.tolist()
    features = ['PM2.5', 'AMB_TEMP', 'RH', 'RAINFALL', 'WIND_DIREC', 'WIND_SPEED', 
                'PM10', 'NO', 'NO2', 'NOx', 'O3', 'CO', 'SO2']
    dataframe = dataframe[features]
    dataframe = dataframe.sort_index()
    dataframe = dataframe.rename(columns={"PM2.5":"PM25", "AMB_TEMP":"TEMP", "WIND_SPEED":"WS", "WIND_DIREC":"WD", "RAINFALL":"RAIN"})
    dataframe.to_csv(my_output)
    